<a href="https://colab.research.google.com/github/Ananyas-7/ai-dyslexia-tool/blob/main/notebooks/data_insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install libraries if not already installed
!pip install pandas matplotlib seaborn wordcloud textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.2/239.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import textstat

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
dataset_path = "/content/drive/MyDrive/data-simplification.tar.bz2"

In [7]:
import os
# Create folder for extraction
extract_dir = "/content/drive/MyDrive/ai-dyslexia-tool/data/raw/wikilarge"
os.makedirs(extract_dir, exist_ok=True)
# Extract using tar
!tar -xvjf "/content/drive/MyDrive/data-simplification.tar.bz2" -C {extract_dir}


data-simplification/
data-simplification/wikilarge/
data-simplification/wikilarge/wiki.full.aner.map.t7
data-simplification/wikilarge/wiki.full.aner.ori.test.dst
data-simplification/wikilarge/wiki.full.aner.ori.test.src
data-simplification/wikilarge/wiki.full.aner.ori.train.dst
data-simplification/wikilarge/wiki.full.aner.ori.train.src
data-simplification/wikilarge/wiki.full.aner.ori.valid.dst
data-simplification/wikilarge/wiki.full.aner.ori.valid.src
data-simplification/wikilarge/wiki.full.aner.test.dst
data-simplification/wikilarge/wiki.full.aner.test.src
data-simplification/wikilarge/wiki.full.aner.train.dst
data-simplification/wikilarge/wiki.full.aner.train.dst.vocab.tmp.t7
data-simplification/wikilarge/wiki.full.aner.train.src
data-simplification/wikilarge/wiki.full.aner.train.src.vocab.tmp.t7
data-simplification/wikilarge/wiki.full.aner.valid.dst
data-simplification/wikilarge/wiki.full.aner.valid.src
data-simplification/wikismall/
data-simplification/wikismall/PWKP_108016.tag.80.